In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
.master("local[*]") \
.appName("test") \
.getOrCreate()

24/04/05 14:17:54 WARN Utils: Your hostname, ahona-ThinkPad-X270-W10DG resolves to a loopback address: 127.0.1.1; using 192.168.43.6 instead (on interface wlp3s0)
24/04/05 14:17:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/05 14:17:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/05 14:17:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/05 14:17:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
!ls

Dockerfile		  ingest_data_pyspark.py  Untitled.ipynb
fhhv			  taxi_zone_lookup.csv	  venv
fhv_tripdata_2021-01.csv  test_01.ipynb		  yellow_data
green_data		  test_03.ipynb		  yellow_data_parquet
green_data_parquet	  test_spark.ipynb	  zones
head.csv		  Untitled1.ipynb


In [5]:
df_green = spark.read.parquet('green_data_parquet/')

In [6]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [7]:
df_yellow = spark.read.parquet('yellow_data_parquet/')

In [8]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [15]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [16]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [17]:
df_green.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [18]:
df_yellow.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [20]:
set(df_yellow.columns) & set(df_green.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [21]:
common_columns = []

yellow_columns = set(df_yellow.columns) 

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [22]:
from pyspark.sql import functions as F

In [24]:
df_green_sel = df_green \
  .select(common_columns) \
  .withColumn('service_type', F.lit('green'))

In [25]:
df_yellow_sel = df_yellow \
  .select(common_columns) \
  .withColumn('service_type', F.lit('yellow'))

In [26]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [27]:
df_trips_data.groupBy('service_type').count().show()

+------------+-------+
|service_type|  count|
+------------+-------+
|       green|1078688|
|      yellow|7667792|
+------------+-------+



In [28]:
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

In [30]:
df_trips_data.createOrReplaceTempView('trips_data')

In [31]:
spark.sql(
    """
    SELECT
        service_type,
        count(1)
    FROM
        trips_data
    GROUP BY 
        service_type
    """
).show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 1078688|
|      yellow| 7667792|
+------------+--------+



In [32]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [35]:
!ls

Dockerfile		  ingest_data_pyspark.py  Untitled1.ipynb
fhhv			  result		  Untitled.ipynb
fhv_tripdata_2021-01.csv  taxi_zone_lookup.csv	  venv
green_data		  test_01.ipynb		  yellow_data
green_data_parquet	  test_03.ipynb		  yellow_data_parquet
head.csv		  test_spark.ipynb	  zones


In [37]:
df_result.coalesce(1).write.parquet('result', mode='overwrite')